# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## GPU Check

First, you'll need to enable GPUs for the notebook:

- Navigate to `Edit`→`Notebook Settings`
- Select T4 GPU from the Hardware Accelerator section
- Click `Save` and accept.

Next, we'll confirm that we can connect to the GPU:

In [1]:
import torch

if not torch.cuda.is_available():
    raise RuntimeError("GPU not available")
else:
    print("GPU is available!")

GPU is available!


## Installing dependencies

In [2]:
%pip install ragatouille PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.9/490.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.0 MB/s eta 0:00:00
 

In [3]:
%pip install git+https://github.com/mozilla-ai/structured-qa.git@5-add-benchmark

  Cloning https://github.com/mozilla-ai/structured-qa.git (to revision 5-add-benchmark) to /tmp/pip-req-build-fztvdq23
  Running command git clone --filter=blob:none --quiet https://github.com/mozilla-ai/structured-qa.git /tmp/pip-req-build-fztvdq23
  Running command git checkout -b 5-add-benchmark --track origin/5-add-benchmark
  Switched to a new branch '5-add-benchmark'
  Branch '5-add-benchmark' set up to track remote branch '5-add-benchmark' from 'origin'.
  Resolved https://github.com/mozilla-ai/structured-qa.git to commit 0b8e5cf9d2db91af71478a715bfdbba1b36316fa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━

In [4]:
!wget https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv

--2025-01-29 09:01:27--  https://raw.githubusercontent.com/mozilla-ai/structured-qa/refs/heads/5-add-benchmark/benchmark/structured_qa.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21734 (21K) [text/plain]
Saving to: ‘structured_qa.csv’

structured_qa.csv   100%[===================>]  21.22K  --.-KB/s    in 0.002s  

2025-01-29 09:01:27 (13.3 MB/s) - ‘structured_qa.csv’ saved [21734/21734]



# Setup

In [5]:
import os
import google.generativeai as genai
from google.colab.userdata import get, SecretNotFoundError

try:
    genai.configure(api_key=get("GOOGLE_API_KEY"))
except SecretNotFoundError as e:
    raise RuntimeError("Please set the GOOGLE_API_KEY secret to your API key") from e
os.environ["LOGURU_LEVEL"] = "INFO"

In [6]:
from loguru import logger

In [7]:
import PyPDF2


def load_pdf(pdf_file: str) -> str | None:
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        return "\n".join(page.extract_text() for page in pdf_reader.pages)
    except Exception as e:
        logger.exception(e)
        return None

## Function to Process all questions for a single Document

In [8]:
import json
import time

from ragatouille import RAGPretrainedModel
from ragatouille.data import CorpusProcessor


def process_document(
    document_file,
    document_data,
    model,
):
    logger.info("Setting up RAG")
    RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")
    corpus_processor = CorpusProcessor()
    documents = corpus_processor.process_corpus([load_pdf(document_file)])
    RAG.encode([x["content"] for x in documents])

    logger.info("Predicting")
    answers = {}
    sections = {}
    for index, row in document_data.iterrows():
        if model.n > 0 and model.n % 9 == 0:
            logger.info("Waiting for 60 seconds")
            time.sleep(60)
        question = row["question"]
        question_part, *options = question.split("?")

        logger.info(f"Question: {question}")
        results = RAG.search_encoded_docs(query=question_part, k=3)
        current_info = "\n".join(result["content"] for result in results)
        logger.info(current_info[:100])

        answer = model.model.generate_content(
            [f"This is the document: {current_info}", question]
        )
        logger.info(answer.text)
        answers[index] = json.loads(answer.text)["answer"]
        sections[index] = None
        model.n += 1

    return answers, sections

## Load Model

In [9]:
from structured_qa.model_loaders import load_gemini_model

In [10]:
SYSTEM_PROMPT = """
You are a rigorous assistant answering questions.
You must only answer based on the current information available which is:

```
{CURRENT_INFO}
```

If the current information available not enough to answer the question,
you must return "I need more info" srting and nothing else:

If the current information is enough to answer, you must return one of the following formats:
- YES/NO (for boolean questions)
- Number (for numeric questions)
- Single letter (for multiple-choice questions)
"""

In [11]:
model = load_gemini_model("gemini-2.0-flash-exp", system_prompt=SYSTEM_PROMPT)
model.n = 0

# Run Benchmark

In [12]:
from pathlib import Path
from urllib.request import urlretrieve

import pandas as pd

logger.info("Loading input data")
data = pd.read_csv("structured_qa.csv")
data["pred_answer"] = [None] * len(data)
data["pred_section"] = [None] * len(data)
for document_link, document_data in data.groupby("document"):
    logger.info(f"Downloading document {document_link}")
    downloaded_document = Path(f"{Path(document_link).name}.pdf")
    if not Path(downloaded_document).exists():
        urlretrieve(document_link, downloaded_document)
        logger.info(f"Downloaded {document_link} to {downloaded_document}")
    else:
        logger.info(f"File {downloaded_document} already exists")

    answers, sections = process_document(downloaded_document, document_data, model)

    for index in document_data.index:
        data.loc[index, "pred_answer"] = str(answers[index]).upper()
        data.loc[index, "pred_section"] = sections[index]

data.to_csv("results.csv")

2025-01-29 09:01:52.976 | INFO     | __main__:<cell line: 0>:6 - Loading input data
2025-01-29 09:01:53.001 | INFO     | __main__:<cell line: 0>:11 - Downloading document https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf
2025-01-29 09:01:53.336 | INFO     | __main__:<cell line: 0>:15 - Downloaded https://aiindex.stanford.edu/wp-content/uploads/2024/05/HAI_AI-Index-Report-2024.pdf to HAI_AI-Index-Report-2024.pdf.pdf
2025-01-29 09:01:53.338 | INFO     | __main__:process_document:13 - Setting up RAG
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional

artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

...

2025-01-29 09:13:16.968 | INFO     | __main__:process_document:37 - {
"answer": 3
}
2025-01-29 09:13:16.969 | INFO     | __main__:process_document:24 - Waiting for 60 seconds
2025-01-29 09:14:16.972 | INFO     | __main__:process_document:29 - Question: How many victory conditions are there?
2025-01-29 09:14:16.989 | INFO     | __main__:process_document:32 - Conquering Middle-earth
If you are present in all 7 regions (with a Fortress and/or at least 1 Unit)
2025-01-29 09:14:18.783 | INFO     | __main__:process_document:37 - {
  "answer": 3
}
2025-01-29 09:14:18.785 | INFO     | __main__:process_document:29 - Question: How many different races are there?
2025-01-29 09:14:18.801 | INFO     | __main__:process_document:32 - • 
 Re
veal tokens to both players. There is no hidden information.Bonus spaces
2 matching Race symb
2025-01-29 09:14:20.417 | INFO     | __main__:process_document:37 - {
  "answer": 7
}
2025-01-29 09:14:20.419 | INFO     | __main__:process_document:29 - Question: Which 

Encoding 48 documents...


  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
100%|██████████| 2/2 [00:00<00:00,  9.92it/s]
2025-01-29 09:15:43.029 | INFO     | __main__:process_document:19 - Predicting
2025-01-29 09:15:43.030 | INFO     | __main__:process_document:29 - Question: What is the maximum number of cards a player may acquire during the lookout phase?
2025-01-29 09:15:43.047 | INFO     | __main__:process_document:32 - 13. At the beginning of the game, each player draws 5 cards from their 
Clan deck, looks through the


Shapes:
encodings: torch.Size([48, 508, 128])
doc_masks: torch.Size([48, 508])
Documents encoded!


2025-01-29 09:15:44.189 | INFO     | __main__:process_document:37 - {
  "answer": 4
}
2025-01-29 09:15:44.192 | INFO     | __main__:process_document:29 - Question: Is there a limit to the number of cards a player may have in their hand?
2025-01-29 09:15:44.213 | INFO     | __main__:process_document:32 - NOTE 1:  There’s no limit to the number of cards a player may have 
in their hand.  
NOTE 2:  If the
2025-01-29 09:15:50.715 | INFO     | __main__:process_document:37 - {
"answer": "No"
}
2025-01-29 09:15:50.716 | INFO     | __main__:process_document:29 - Question: Can you raid the locations of a player that has passed during the action phase?
2025-01-29 09:15:50.735 | INFO     | __main__:process_document:32 - Starting with the First player and continuing clockwise, 
each player performs one action at a time.
2025-01-29 09:15:52.202 | INFO     | __main__:process_document:37 - {
  "answer": "No"
}
2025-01-29 09:15:52.203 | INFO     | __main__:process_document:29 - Question: How many poin

In [13]:
results = pd.read_csv("results.csv")
results.loc[results["answer"] != results["pred_answer"]]

,Unnamed: 0,document,section,question,answer,pred_answer,pred_section
22,22,https://eur-lex.europa.eu/legal-content/EN/TXT...,Classification of general-purpose AI models as...,"What is the threshold, measured in floating po...",C,NO SPECIFIC THRESHOLD IS MENTIONED IN THE DOCU...,NaN
44,44,https://arxiv.org/pdf/2201.11903,3.2 Results,How many random samples were examined to under...,100,50,NaN
47,47,https://github.com/mozilla-ai/structured-qa/re...,CARD AND TILE EFFECTS,How many different races are there?,6,7,NaN
52,52,https://github.com/mozilla-ai/structured-qa/re...,CARD AND TILE COSTS,Can a player pay coins to compensate for missi...,YES,NO,NaN
55,55,https://github.com/mozilla-ai/structured-qa/re...,CARD AND TILE EFFECTS,Which type of cards provide coins? -A: Gray -B...,B,NONE OF THE ABOVE,NaN
66,66,https://github.com/mozilla-ai/structured-qa/re...,EXPEDITION PHASE,How many victory points you get from each conq...,1,THE DOCUMENT SAYS THAT PLAYERS GAIN VPS FROM P...,NaN
83,83,https://docs.nvidia.com/cuda/pdf/CUDA_C_Progra...,15.3. API Fundamentals,When are virtual addresses assigned to graph a...,C,A,NaN


In [14]:
accuracy = sum(results["answer"] == results["pred_answer"]) / len(results)
accuracy

0.9292929292929293